In [1]:
!git clone https://github.com/clarkkev/attention-analysis.git

Cloning into 'attention-analysis'...
remote: Enumerating objects: 48, done.
remote: Total 48 (delta 0), reused 0 (delta 0), pack-reused 48
Receiving objects: 100% (48/48), 422.25 KiB | 7.68 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [ ]:
import json
import utils
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from bert import modeling
from bert import tokenization
import bpe_utils
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
train = pd.read_csv("./data/medical_tc_train_cleaned.csv")
test =  pd.read_csv("./data/medical_tc_test_cleaned.csv")
labels = pd.read_csv("./data/medical_tc_labels.csv")
mappings = {
    0: "non-cardiovascular",
    1: "cardiovascular"
}
inverse_mapping = {
    "non-cardiovascular" : 0,
    "cardiovascular" : 1
}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
model = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
num_classes = 2
model.classifier = nn.Sequential(
    nn.Linear(model.config.hidden_size, 256),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(256, num_classes)
)
model.to('cuda')

with open('./attention-analysis/vocab.txt', 'w') as f:
    for token in tokenizer.vocab.keys():
        f.write(token+'\n')

model.load_state_dict(torch.load("./attention-analysis/biobert_fine_tuned_epoch_2.pt")['model_state_dict'])

In [ ]:
data = [
    'case unilateral posterior ischemic optic neuropathy after radical neck dissection we present case unilateral posterior ischemic optic neuropathy after bilateral radical neck dissection etiologic factors discussed',
    'lipomatous hypertrophy interatrial septum unusual intraoperative finding lipomatous hypertrophy interatrial septum incidental finding man 45 undergoing coronary artery bypass grafting unstable angina he not overweight and did not have any rhythm disturbances diagnosis made frozen section',
    'atheroemboli lower urinary tract marker atherosclerotic vascular diseasea case report atheroemboli lower genitourinary tract may serve as marker disseminated atheroembolic disease highly lethal condition case presentation and review our institional experience presented',
    'hydatid cyst interatrial and interventricular septum heart our knowledge this report describes first case large hydatid cyst involving full thickness both interatrial and interventricular septum heart which detected at autopsy cyst intact clinically there complete heart block',
    'cavernous hemangioma spinal cord report 2 unusual cases we present 2 cases spinal cord intramedullary cavernous hemangioma 1 patient 1st reported case multiple spinal cord lesions diagnosis greatly enhanced by use mri',
    'captoprilinduced cholestatic jaundice i have reported case captoprilinduced cholestatic jaundice this drug being used with increasing frequency so it important that physicians recognize this adverse effect captoprilinduced jaundice resolves after cessation captopril therapy',
    'supravalvar aortic stenosis complication aortic valve surgery after patient had aortic valvotomy and aortotomy relief valvar aortic stenosis supravalvar stenosis developed over next six years',
    'giant left atriuma case report seventysevenyear old woman with mitral stenosis presented with cardiomegaly evident her chest roentgenogram cardiac enlargement due giant left atrium that distorted cardiac structures echocardiogram and firstpass nuclear angiogram able delineate huge left atrium',
    'management brucella endocarditis with aortic root abscess three cases brucella endocarditis with aortic root abscess reported two patients successfully managed by combination medical therapy and surgery third patient died suddenly 36 hours after admission hospital',
    'rib compression coronary arteries this report describes finding coronary artery narrowing caused by compression by overlying rib two patients with cardiomegaly there probably no clinical significance this finding primary differential diagnostic entity myocardial bridging',
    'dynamic cardiomyoplasty chronic chagas heart disease clinicopathological data we report 44yearold man with chronic chagasic cardiomyopathy who underwent latissimus dorsi dynamic cardiomyoplasty and died 4 months later clinicopathological findings discussed and literature reviewed',
    'autonomic dysfunction and guillainbarre syndrome use esmolol its management 17yearold girl with guillainbarre syndrome and autonomic dysfunction treated successfully with esmolol esmolol may be appropriate drug rapid assessment and control tachyarrhythmias critically ill patients',
    'vagal schwannoma associated with syncopal attacks and postural hypotension case report case vagal schwannoma cerebellomedullary angle reported preoperatively patient had paroxysmal episodes postural hypotension with syncope after total removal vagal tumor her systemic blood pressure returned normal',
    'ptca gastroepiploic bypass technique percutaneous transluminal coronary angioplasty ptca gastroepiploic bypass described using standard ptca devices severe spasm gastroepiploic bypass occurred modification guide catheter position suggested avoid inducing gastroepiploic bypass spasm',
    'catheterization coronary artery bypass graft from descending aorta increasing frequency reoperation coronary artery disease has led use variety grafts this report describes catheter technique selective opacification saphenous vein graft from descending thoracic aorta posterior coronary circulation',
    'gastroesophageal reflux infants clinically significant gastroesophageal reflux occurs one 500 infants symptoms resolve 60 80 percent infants by 18 months age thickened formula and position changes often recommended medication and surgical intervention may be needed refractory cases',
    'sudden death from acute cocaine intoxication virginia 1988 review medical examiners autopsy records revealed that 1988 33 residents virginia died acute cocaine intoxication subjects 19 45 years old most men used cocaine intravenously and used other toxic substances with lethal dose cocaine',
    'atrial fibrillation elderly management update nonvalvular atrial fibrillation associated with markedly increased risk embolic stroke elderly persons evidence accumulating that anticoagulation with warfarin or aspirin may be effective reducing this risk',
    'false aneurysm internal mammary artery iatrogenic false aneurysm branch internal mammary artery after sternal wire closure very rare occurrence we describe case which appeared 62 year old woman operated upon mitral valve replacement and diagnosed after digital subtraction angiography',
    'meningitis due protozoa and helminths this article reviews microbiology pathogenesis epidemiology clinical manifestations diagnostic tests and recent advances therapy protozoan and helminthic infections central nervous system with more emphasis given protozoan than helminthic infections',
    'indomethacin responsive hypercalcaemia associated with renal sarcoma infant presented with nonmetastatic renal spindle cell sarcoma and hypercalcaemia which resolved after treatment with indomethacin there vivo and vitro evidence that hypercalcaemia mediated by circulatory prostaglandins',
    'ten cases transitional cell carcinoma bladder causing ureteric obstruction review carried out 10 patients with superficial transitional cell carcinoma bladder ta lesions that causing ureteric obstruction evidence upper tract obstruction did not necessarily indicate deep invasion',
    'preventing colorectal cancer knowledgeable patients should not die colorectal cancer increasing intake dietary fiber decreasing fat consumption and increasing use modern technology detect adenomatous polyps and early cancer can greatly decrease mortality associated with colorectal cancer'
]

In [ ]:
input_json = []
for d in data:
    temp = {}
    temp['text'] = d
    temp['words'] = d.split()
    input_json.append(temp)

with open('./attention-analysis/input.json', 'w') as f:
    json.dump(input_json, f)

with open('./attention-analysis/model_config.json', 'w') as f:
    json.dump(model.config.to_dict(), f)

PATH = "./attention-analysis/model.pt"
LOSS = 0.0
torch.save({
            'epoch': 4,
            'model_state_dict': model.state_dict(),
            'loss': LOSS,
            }, PATH)

In [ ]:
class Example(object):
  """Represents a single input sequence to be passed into BERT."""

  def __init__(self, features, tokenizer, max_sequence_length,):
    self.features = features

    if "tokens" in features:
      self.tokens = features["tokens"]
    else:
      if "text" in features:
        text = features["text"]
      else:
        text = " ".join(features["words"])
      self.tokens = ["[CLS]"] + tokenizer.tokenize(text) + ["[SEP]"]
    # print(self.tokens)
    self.input_ids = tokenizer.convert_tokens_to_ids(self.tokens)
    self.segment_ids = [0] * len(self.tokens)
    self.input_mask = [1] * len(self.tokens)
    while len(self.input_ids) < max_sequence_length:
      self.input_ids.append(0)
      self.input_mask.append(0)
      self.segment_ids.append(0)


def examples_in_batches(examples, batch_size):
  for i in utils.logged_loop(range(1 + ((len(examples) - 1) // batch_size))):
    yield examples[i * batch_size:(i + 1) * batch_size]


class AttnMapExtractor(object):
  """Runs BERT over examples to get its attention maps."""

  def __init__(self, bert_config_file, init_checkpoint,
               max_sequence_length=128, debug=False):
    make_placeholder = lambda name: tf.placeholder(
        tf.int32, shape=[None, max_sequence_length], name=name)
    self._input_ids = make_placeholder("input_ids")
    self._segment_ids = make_placeholder("segment_ids")
    self._input_mask = make_placeholder("input_mask")

    bert_config = modeling.BertConfig.from_json_file(bert_config_file)
    if debug:
      bert_config.num_hidden_layers = 12
      bert_config.hidden_size = 144

    self._attn_maps = modeling.BertModel(
        config=bert_config,
        is_training=False,
        input_ids=self._input_ids,
        input_mask=self._input_mask,
        token_type_ids=self._segment_ids,
        use_one_hot_embeddings=True).attn_maps

    if not debug:
      print("Loading BERT from checkpoint...")
      assignment_map, _ = modeling.get_assignment_map_from_checkpoint(
          tf.trainable_variables(), init_checkpoint)
      tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

  def get_attn_maps(self, sess, examples):
    feed = {
        self._input_ids: np.vstack([e.input_ids for e in examples]),
        self._segment_ids: np.vstack([e.segment_ids for e in examples]),
        self._input_mask: np.vstack([e.input_mask for e in examples])
    }
    return sess.run(self._attn_maps, feed_dict=feed)


In [ ]:
examples = []
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
model = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
num_classes = 2
model.classifier = nn.Sequential(
    nn.Linear(model.config.hidden_size, 256),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(256, num_classes)
)
model.to('cuda')
model.load_state_dict(torch.load("./biobert_fine_tuned_epoch_16.pt")['model_state_dict'])

for features in utils.load_json('./input.json'):
  print(features)
  example = Example(features, tokenizer, 256)
  if len(example.input_ids) <= 256:
    examples.append(example)


In [ ]:
print("Building BERT model...")
extractor = AttnMapExtractor('./model_config.json','./model.pt',256,True)

print("Extracting attention maps...")
feature_dicts_with_attn = []
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for batch_of_examples in examples_in_batches(examples, 8):
    attns = extractor.get_attn_maps(sess, batch_of_examples)
    for e, e_attn in zip(batch_of_examples, attns):
      seq_len = len(e.tokens)
      e.features["attns"] = e_attn[:, :, :seq_len, :seq_len].astype("float16")
      e.features["tokens"] = e.tokens
      feature_dicts_with_attn.append(e.features)

outpath = './model_attention.pkl'
print("Writing attention maps to {:}...".format(outpath))
utils.write_pickle(feature_dicts_with_attn, outpath)

data = utils.load_pickle('./model_attention.pkl')


In [ ]:
print("Computing head distances")
js_distances = np.zeros([144, 144])
for doc in utils.logged_loop(data, n_steps=None):
    if "attns" not in doc:
        continue
    tokens, attns = doc["tokens"], np.array(doc["attns"])

    attns_flat = attns.reshape([144, attns.shape[2], attns.shape[3]])
    for head in range(144):
        head_attns = np.expand_dims(attns_flat[head], 0)
        head_attns_smoothed = (0.001 / head_attns.shape[1]) + (head_attns * 0.999)
        attns_flat_smoothed = (0.001 / attns_flat.shape[1]) + (attns_flat * 0.999)
        m = (head_attns_smoothed + attns_flat_smoothed) / 2
        js = -head_attns_smoothed * np.log(m / head_attns_smoothed)
        js += -attns_flat_smoothed * np.log(m / attns_flat_smoothed)
        js /= 2
        js = js.sum(-1).sum(-1)
        js_distances[head] += js

    utils.write_pickle(js_distances, './head_distances.pkl')
